# Detecção de Outliers em Cybersecurity: Análise de Ameaças usando Aprendizado Não Supervisionado

## Resumo Executivo

Este notebook implementa e avalia técnicas de aprendizado não supervisionado para detecção de ameaças cibernéticas através da identificação de outliers. O objetivo principal é validar se os outliers detectados pelos algoritmos correspondem efetivamente a agentes maliciosos no dataset Text-Based Cyber Threat Detection.

### Objetivos:
1. Aplicar técnicas de clustering para identificar padrões nos dados
2. Utilizar algoritmos de detecção de outliers (Isolation Forest, LOF, One-Class SVM)
3. Validar os outliers detectados contra labels reais de ameaças
4. Comparar a eficácia dos diferentes métodos

### Metodologia:
- Dataset: Text-Based Cyber Threat Detection (Kaggle)
- Técnicas: Clustering (K-Means, DBSCAN) + Detecção de Outliers (IF, LOF, OC-SVM)
- Validação: Métricas de classificação (Precision, Recall, F1-Score)

---

**Projeto**: Mitigação de Ataques por Envenenamento em Aprendizado Federado  
**Instituição**: Faculdade Impacta  
**Data**: Outubro 2025

## 1. Setup e Configuração do Ambiente

Importação de bibliotecas necessárias e configuração do ambiente de visualização.

In [ ]:
# Importação de bibliotecas padrão
import pandas as pd
import numpy as np
import os
import warnings
from datetime import datetime

# Visualização
import matplotlib.pyplot as plt
import seaborn as sns

# Scikit-learn: Pré-processamento
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import PCA

# Scikit-learn: Clustering
from sklearn.cluster import KMeans, DBSCAN, AgglomerativeClustering
from sklearn.metrics import silhouette_score, calinski_harabasz_score

# Scikit-learn: Detecção de Outliers
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor
from sklearn.svm import OneClassSVM
from sklearn.covariance import EllipticEnvelope

# Scikit-learn: Métricas de avaliação
from sklearn.metrics import (
    classification_report, confusion_matrix, 
    accuracy_score, precision_score, recall_score, f1_score,
    roc_auc_score, roc_curve, precision_recall_curve
)

# Configurações
warnings.filterwarnings('ignore')
np.random.seed(42)

# Configurar visualizações
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

# Criar diretório de saída para imagens
OUTPUT_DIR = os.path.join(os.getcwd(), 'output_images')
os.makedirs(OUTPUT_DIR, exist_ok=True)

print("Ambiente configurado com sucesso!")
print(f"Diretório de saída: {OUTPUT_DIR}")

[## 2. Carregamento e Exploração Inicial dos Dados

IMPORTANTE: Este notebook assume que você baixou o dataset do Kaggle.  
Dataset: https://www.kaggle.com/datasets/ramoliyafenil/text-based-cyber-threat-detection

Coloque o arquivo CSV na pasta `data/` do projeto., ## 3. Análise Exploratória dos Dados, ## 4. Pré-processamento e Feature Engineering, ## 5. Detecção de Outliers - Múltiplas Técnicas, ## 6. Validação: Comparação com Labels Reais, ## 7. Visualização de Resultados, ## 8. Avaliação e Conclusões]

## 2. Carregamento e Exploração Inicial dos Dados

O dataset foi baixado do Kaggle usando kagglehub e está localizado em `../../data/cyber-outlier-detection/`.

**Dataset**: Text-Based Cyber Threat Detection  
**Amostras**: 19,940 registros  
**Features**: 10 colunas (text, entities, relations, labels, etc.)

In [ ]:
# Carregar dataset principal
data_path = '../../data/cyber-outlier-detection/cyber-threat-intelligence_all.csv'

print("Carregando dataset...")
df = pd.read_csv(data_path)

print(f"✓ Dataset carregado com sucesso!")
print(f"  Shape: {df.shape}")
print(f"  Memória: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

# Visualizar primeiras linhas
print("\nPrimeiras 5 linhas:")
df.head()

In [ ]:
# Informações gerais do dataset
print("="*80)
print("ANÁLISE EXPLORATÓRIA")
print("="*80)

print("\n1. INFORMAÇÕES GERAIS:")
print(f"   Total de registros: {len(df):,}")
print(f"   Total de colunas: {len(df.columns)}")

print("\n2. COLUNAS E TIPOS:")
for i, col in enumerate(df.columns, 1):
    non_null = df[col].notna().sum()
    null_pct = (df[col].isna().sum() / len(df)) * 100
    print(f"   {i:2d}. {col:20s} - {str(df[col].dtype):10s} ({non_null:5d} não-nulos, {null_pct:5.1f}% nulos)")

print("\n3. ESTATÍSTICAS DAS COLUNAS PRINCIPAIS:")

# Coluna text
if 'text' in df.columns:
    print(f"\n   TEXT:")
    print(f"      Textos únicos: {df['text'].nunique():,}")
    print(f"      Comprimento médio: {df['text'].str.len().mean():.0f} caracteres")
    print(f"      Comprimento mín/máx: {df['text'].str.len().min()}/{df['text'].str.len().max()}")

# Coluna label
if 'label' in df.columns:
    print(f"\n   LABELS:")
    print(f"      Labels únicos: {df['label'].nunique()}")
    print(f"      Labels não-nulos: {df['label'].notna().sum():,} ({(df['label'].notna().sum()/len(df))*100:.1f}%)")
    print(f"\n      Top 10 categorias:")
    print(df['label'].value_counts().head(10).to_string())

# Coluna entities
if 'entities' in df.columns:
    print(f"\n   ENTITIES:")
    non_null_entities = df['entities'].notna().sum()
    print(f"      Registros com entities: {non_null_entities:,} ({(non_null_entities/len(df))*100:.1f}%)")

## 3. Visualizações Exploratórias

Vamos criar visualizações para entender melhor a distribuição dos dados.

In [ ]:
# Criar visualizações exploratórias
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# 1. Distribuição de comprimento de texto
ax1 = axes[0, 0]
text_lengths = df['text'].str.len()
ax1.hist(text_lengths, bins=50, color='skyblue', edgecolor='black', alpha=0.7)
ax1.set_xlabel('Comprimento do Texto (caracteres)', fontsize=12)
ax1.set_ylabel('Frequência', fontsize=12)
ax1.set_title('Distribuição do Comprimento dos Textos', fontsize=14, fontweight='bold')
ax1.axvline(text_lengths.mean(), color='red', linestyle='--', linewidth=2, label=f'Média: {text_lengths.mean():.0f}')
ax1.legend()
ax1.grid(alpha=0.3)

# 2. Top 15 labels (excluindo nulos)
ax2 = axes[0, 1]
df_labels = df['label'].dropna()
top_labels = df_labels.value_counts().head(15)
bars = ax2.barh(range(len(top_labels)), top_labels.values, color='coral', edgecolor='black')
ax2.set_yticks(range(len(top_labels)))
ax2.set_yticklabels(top_labels.index, fontsize=10)
ax2.set_xlabel('Contagem', fontsize=12)
ax2.set_title('Top 15 Categorias de Labels', fontsize=14, fontweight='bold')
ax2.invert_yaxis()
ax2.grid(axis='x', alpha=0.3)

# Adicionar valores nas barras
for i, (bar, value) in enumerate(zip(bars, top_labels.values)):
    ax2.text(value + 20, i, f'{value}', va='center', fontsize=9)

# 3. Distribuição de valores nulos
ax3 = axes[1, 0]
null_counts = df.isnull().sum()
null_pct = (null_counts / len(df)) * 100
columns_with_nulls = null_pct[null_pct > 0].sort_values(ascending=True)

if len(columns_with_nulls) > 0:
    bars = ax3.barh(range(len(columns_with_nulls)), columns_with_nulls.values, color='salmon', edgecolor='black')
    ax3.set_yticks(range(len(columns_with_nulls)))
    ax3.set_yticklabels(columns_with_nulls.index, fontsize=10)
    ax3.set_xlabel('Percentual de Valores Nulos (%)', fontsize=12)
    ax3.set_title('Valores Nulos por Coluna', fontsize=14, fontweight='bold')
    ax3.grid(axis='x', alpha=0.3)
    
    for i, (bar, value) in enumerate(zip(bars, columns_with_nulls.values)):
        ax3.text(value + 1, i, f'{value:.1f}%', va='center', fontsize=9)
else:
    ax3.text(0.5, 0.5, 'Sem valores nulos', ha='center', va='center', fontsize=14)
    ax3.set_xlim(0, 1)
    ax3.set_ylim(0, 1)

# 4. Proporção de registros com/sem labels
ax4 = axes[1, 1]
label_status = df['label'].notna().value_counts()
colors = ['#66c2a5', '#fc8d62']
explode = (0.05, 0)
wedges, texts, autotexts = ax4.pie(label_status.values, 
                                     labels=['Com Label', 'Sem Label'],
                                     colors=colors,
                                     autopct='%1.1f%%',
                                     startangle=90,
                                     explode=explode,
                                     textprops={'fontsize': 12, 'fontweight': 'bold'})
ax4.set_title('Proporção de Registros com/sem Labels', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.savefig(os.path.join(OUTPUT_DIR, '01_exploratory_analysis.png'), dpi=300, bbox_inches='tight')
plt.show()

print(f"✓ Visualização salva: {OUTPUT_DIR}/01_exploratory_analysis.png")

## 4. Pré-processamento e Feature Engineering

Para aplicar técnicas de detecção de outliers, precisamos converter o texto em features numéricas. Vamos usar:
1. **TF-IDF** para vetorizar o texto
2. **PCA** para reduzir dimensionalidade
3. **StandardScaler** para normalizar features

### 4.1. Preparação dos Dados com Labels

In [ ]:
# Filtrar apenas registros com labels para validação
df_labeled = df[df['label'].notna()].copy()

print(f"Dataset com labels: {len(df_labeled):,} registros")
print(f"Dataset completo: {len(df):,} registros")
print(f"Percentual com labels: {(len(df_labeled)/len(df))*100:.1f}%")

# Criar labels binários: threat (malware, attack-pattern, threat-actor, vulnerability) vs normal
threat_labels = ['malware', 'attack-pattern', 'threat-actor', 'vulnerability', 'tools']

df_labeled['is_threat'] = df_labeled['label'].apply(
    lambda x: 1 if str(x).lower() in threat_labels else 0
)

print(f"\nDistribuição de ameaças:")
print(f"  Threats (1): {(df_labeled['is_threat'] == 1).sum():,} ({(df_labeled['is_threat'] == 1).sum()/len(df_labeled)*100:.1f}%)")
print(f"  Normal  (0): {(df_labeled['is_threat'] == 0).sum():,} ({(df_labeled['is_threat'] == 0).sum()/len(df_labeled)*100:.1f}%)")

# Amostra representativa para análise (para performance)
# Vamos usar uma amostra estratificada de 15,000 registros
sample_size = min(15000, len(df_labeled))
df_sample = df_labeled.sample(n=sample_size, random_state=42, stratify=df_labeled['is_threat'])

print(f"\nUsando amostra de {len(df_sample):,} registros para análise")
print(f"  Threats: {(df_sample['is_threat'] == 1).sum():,}")
print(f"  Normal:  {(df_sample['is_threat'] == 0).sum():,}")

### 4.2. Vetorização TF-IDF

Converter textos em vetores numéricos usando TF-IDF (Term Frequency-Inverse Document Frequency).

In [ ]:
print("="*80)
print("VETORIZAÇÃO TF-IDF")
print("="*80)

# TF-IDF Vectorizer com parâmetros ajustados
vectorizer = TfidfVectorizer(
    max_features=500,           # Top 500 features mais importantes
    min_df=5,                   # Palavra deve aparecer em no mínimo 5 documentos
    max_df=0.7,                 # Ignorar palavras que aparecem em mais de 70% dos docs
    ngram_range=(1, 2),         # Unigramas e bigramas
    stop_words='english',       # Remover stopwords
    strip_accents='unicode'
)

# Aplicar vetorização
print("\nVetorizando textos...")
X_tfidf = vectorizer.fit_transform(df_sample['text'])

print(f"✓ Vetorização completa!")
print(f"  Shape da matriz TF-IDF: {X_tfidf.shape}")
print(f"  Total de features: {X_tfidf.shape[1]:,}")
print(f"  Sparsity: {(1.0 - X_tfidf.nnz / (X_tfidf.shape[0] * X_tfidf.shape[1])) * 100:.2f}%")

# Converter para array denso
X_tfidf_dense = X_tfidf.toarray()

# Labels verdadeiros
y_true = df_sample['is_threat'].values

print(f"\n  Labels shape: {y_true.shape}")
print(f"  Threats (1): {(y_true == 1).sum():,}")
print(f"  Normal (0):  {(y_true == 0).sum():,}")

### 4.3. Redução de Dimensionalidade com PCA

Reduzir de 500 features para 50 componentes principais (mantendo ~90% da variância).

In [ ]:
print("="*80)
print("REDUÇÃO DE DIMENSIONALIDADE COM PCA")
print("="*80)

# PCA com 50 componentes
n_components = 50
pca = PCA(n_components=n_components, random_state=42)

print(f"\nAplicando PCA ({n_components} componentes)...")
X_pca = pca.fit_transform(X_tfidf_dense)

# Calcular variância explicada
explained_var = pca.explained_variance_ratio_.sum()

print(f"✓ PCA completa!")
print(f"  Shape após PCA: {X_pca.shape}")
print(f"  Variância explicada: {explained_var*100:.2f}%")
print(f"  Redução de dimensionalidade: {X_tfidf_dense.shape[1]} → {X_pca.shape[1]} features")

# Normalizar dados
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_pca)

print(f"\n✓ Dados normalizados com StandardScaler")
print(f"  Mean: {X_scaled.mean():.2e}")
print(f"  Std:  {X_scaled.std():.2f}")

## 5. Detecção de Outliers - Múltiplas Técnicas

Vamos aplicar 5 técnicas diferentes de detecção de outliers e comparar os resultados:

1. **Isolation Forest** - Detecta anomalias isolando observações
2. **Local Outlier Factor (LOF)** - Baseado em densidade local
3. **One-Class SVM** - SVM com kernel RBF para detecção de outliers
4. **Elliptic Envelope** - Assume distribuição gaussiana multivariada
5. **DBSCAN** - Clustering baseado em densidade (pontos em regiões de baixa densidade são outliers)

### 5.1. Isolation Forest

In [ ]:
print("="*80)
print("DETECÇÃO DE OUTLIERS - MÚLTIPLAS TÉCNICAS")
print("="*80)

# Dicionário para armazenar resultados
outlier_predictions = {}
execution_times = {}

# Percentual de contaminação esperado (baseado na proporção de threats)
contamination = (y_true == 1).sum() / len(y_true)
print(f"\nContaminação estimada (threats): {contamination:.3f} ({contamination*100:.1f}%)\n")

import time

# 1. ISOLATION FOREST
print("1. Isolation Forest...")
start = time.time()
iso_forest = IsolationForest(
    contamination=contamination,
    random_state=42,
    n_estimators=100,
    max_samples='auto',
    n_jobs=-1
)
pred_iso = iso_forest.fit_predict(X_scaled)
outlier_predictions['Isolation Forest'] = (pred_iso == -1).astype(int)
execution_times['Isolation Forest'] = time.time() - start
print(f"   ✓ Outliers detectados: {(pred_iso == -1).sum():,} ({(pred_iso == -1).sum()/len(pred_iso)*100:.1f}%)")
print(f"   ✓ Tempo: {execution_times['Isolation Forest']:.2f}s")

# 2. LOCAL OUTLIER FACTOR
print("\n2. Local Outlier Factor (LOF)...")
start = time.time()
lof = LocalOutlierFactor(
    contamination=contamination,
    n_neighbors=20,
    n_jobs=-1
)
pred_lof = lof.fit_predict(X_scaled)
outlier_predictions['LOF'] = (pred_lof == -1).astype(int)
execution_times['LOF'] = time.time() - start
print(f"   ✓ Outliers detectados: {(pred_lof == -1).sum():,} ({(pred_lof == -1).sum()/len(pred_lof)*100:.1f}%)")
print(f"   ✓ Tempo: {execution_times['LOF']:.2f}s")

# 3. ONE-CLASS SVM
print("\n3. One-Class SVM...")
start = time.time()
oc_svm = OneClassSVM(
    nu=contamination,
    kernel='rbf',
    gamma='auto'
)
pred_ocsvm = oc_svm.fit_predict(X_scaled)
outlier_predictions['One-Class SVM'] = (pred_ocsvm == -1).astype(int)
execution_times['One-Class SVM'] = time.time() - start
print(f"   ✓ Outliers detectados: {(pred_ocsvm == -1).sum():,} ({(pred_ocsvm == -1).sum()/len(pred_ocsvm)*100:.1f}%)")
print(f"   ✓ Tempo: {execution_times['One-Class SVM']:.2f}s")

# 4. ELLIPTIC ENVELOPE
print("\n4. Elliptic Envelope...")
start = time.time()
ee = EllipticEnvelope(
    contamination=contamination,
    random_state=42,
    support_fraction=0.9
)
pred_ee = ee.fit_predict(X_scaled)
outlier_predictions['Elliptic Envelope'] = (pred_ee == -1).astype(int)
execution_times['Elliptic Envelope'] = time.time() - start
print(f"   ✓ Outliers detectados: {(pred_ee == -1).sum():,} ({(pred_ee == -1).sum()/len(pred_ee)*100:.1f}%)")
print(f"   ✓ Tempo: {execution_times['Elliptic Envelope']:.2f}s")

# 5. DBSCAN
print("\n5. DBSCAN (Clustering + Outliers)...")
start = time.time()
dbscan = DBSCAN(
    eps=3.0,
    min_samples=10,
    n_jobs=-1
)
pred_dbscan = dbscan.fit_predict(X_scaled)
outlier_predictions['DBSCAN'] = (pred_dbscan == -1).astype(int)
execution_times['DBSCAN'] = time.time() - start
n_clusters = len(set(pred_dbscan)) - (1 if -1 in pred_dbscan else 0)
print(f"   ✓ Clusters encontrados: {n_clusters}")
print(f"   ✓ Outliers detectados: {(pred_dbscan == -1).sum():,} ({(pred_dbscan == -1).sum()/len(pred_dbscan)*100:.1f}%)")
print(f"   ✓ Tempo: {execution_times['DBSCAN']:.2f}s")

print("\n" + "="*80)
print("✓ TODAS AS TÉCNICAS EXECUTADAS COM SUCESSO!")
print("="*80)

## 6. Validação: Comparação com Labels Reais

Agora vamos validar se os outliers detectados correspondem às ameaças reais (threats) no dataset.

**Hipótese**: Outliers detectados = Ameaças cibernéticas (malware, ataques, vulnerabilidades)

Métricas usadas:
- **Accuracy**: % de predições corretas
- **Precision**: % de outliers detectados que são realmente threats
- **Recall**: % de threats que foram detectados como outliers
- **F1-Score**: Média harmônica entre Precision e Recall

In [ ]:
print("="*80)
print("VALIDAÇÃO CONTRA LABELS REAIS")
print("="*80)

# Calcular métricas para cada método
results = []

for method_name, y_pred in outlier_predictions.items():
    # Métricas
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, zero_division=0)
    recall = recall_score(y_true, y_pred, zero_division=0)
    f1 = f1_score(y_true, y_pred, zero_division=0)
    
    # Confusion matrix
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    
    results.append({
        'Método': method_name,
        'Accuracy': accuracy,
        'Precision': precision,
        'Recall': recall,
        'F1-Score': f1,
        'TP': tp,
        'FP': fp,
        'TN': tn,
        'FN': fn,
        'Tempo (s)': execution_times[method_name]
    })
    
    print(f"\n{method_name}:")
    print(f"  Accuracy:  {accuracy*100:.2f}%")
    print(f"  Precision: {precision*100:.2f}%")
    print(f"  Recall:    {recall*100:.2f}%")
    print(f"  F1-Score:  {f1*100:.2f}%")
    print(f"  Confusion Matrix: TP={tp}, FP={fp}, TN={tn}, FN={fn}")

# Criar DataFrame com resultados
df_results = pd.DataFrame(results)
df_results = df_results.sort_values('F1-Score', ascending=False).reset_index(drop=True)

print("\n" + "="*80)
print("RANKING DOS MÉTODOS (por F1-Score):")
print("="*80)
print(df_results[['Método', 'Accuracy', 'Precision', 'Recall', 'F1-Score', 'Tempo (s)']].to_string(index=False))

# Salvar resultados
results_path = os.path.join(OUTPUT_DIR, 'evaluation_metrics.csv')
df_results.to_csv(results_path, index=False)
print(f"\n✓ Resultados salvos: {results_path}")

## 7. Visualização de Resultados

### 7.1. Comparação de Métricas entre Métodos

In [ ]:
# Visualização comparativa de métricas
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

metrics = ['Accuracy', 'Precision', 'Recall', 'F1-Score']
colors = ['#3498db', '#e74c3c', '#2ecc71', '#f39c12']

for idx, metric in enumerate(metrics):
    ax = axes[idx // 2, idx % 2]
    
    # Ordenar por métrica atual
    df_sorted = df_results.sort_values(metric, ascending=True)
    
    # Criar barras horizontais
    bars = ax.barh(range(len(df_sorted)), df_sorted[metric]*100, color=colors[idx], edgecolor='black', alpha=0.8)
    ax.set_yticks(range(len(df_sorted)))
    ax.set_yticklabels(df_sorted['Método'], fontsize=11)
    ax.set_xlabel(f'{metric} (%)', fontsize=12, fontweight='bold')
    ax.set_title(f'{metric} por Método', fontsize=14, fontweight='bold')
    ax.grid(axis='x', alpha=0.3)
    
    # Adicionar valores nas barras
    for i, (bar, value) in enumerate(zip(bars, df_sorted[metric]*100)):
        ax.text(value + 1, i, f'{value:.1f}%', va='center', fontsize=10, fontweight='bold')
    
    # Destacar melhor método
    best_idx = df_sorted[metric].idxmax()
    bars[list(df_sorted.index).index(best_idx)].set_edgecolor('gold')
    bars[list(df_sorted.index).index(best_idx)].set_linewidth(3)

plt.tight_layout()
plt.savefig(os.path.join(OUTPUT_DIR, '02_comparative_metrics.png'), dpi=300, bbox_inches='tight')
plt.show()

print(f"✓ Visualização salva: {OUTPUT_DIR}/02_comparative_metrics.png")

### 7.2. Matrizes de Confusão

In [ ]:
# Visualizar matrizes de confusão
fig, axes = plt.subplots(2, 3, figsize=(18, 10))
axes = axes.ravel()

for idx, (method_name, y_pred) in enumerate(outlier_predictions.items()):
    ax = axes[idx]
    
    # Calcular matriz de confusão
    cm = confusion_matrix(y_true, y_pred)
    
    # Plotar heatmap
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False, ax=ax,
                square=True, linewidths=2, linecolor='black',
                annot_kws={'size': 14, 'weight': 'bold'})
    
    ax.set_xlabel('Predito', fontsize=12, fontweight='bold')
    ax.set_ylabel('Real', fontsize=12, fontweight='bold')
    ax.set_title(f'{method_name}\n(F1: {df_results[df_results["Método"]==method_name]["F1-Score"].values[0]*100:.1f}%)', 
                 fontsize=13, fontweight='bold')
    ax.set_xticklabels(['Normal', 'Threat'], fontsize=11)
    ax.set_yticklabels(['Normal', 'Threat'], fontsize=11, rotation=0)

# Remover subplot extra
fig.delaxes(axes[5])

plt.tight_layout()
plt.savefig(os.path.join(OUTPUT_DIR, '04_confusion_matrices.png'), dpi=300, bbox_inches='tight')
plt.show()

print(f"✓ Visualização salva: {OUTPUT_DIR}/04_confusion_matrices.png")

### 7.3. Comparação de Outliers Detectados vs Ameaças Reais

In [ ]:
# Comparação visual de outliers detectados vs ameaças reais
fig, ax = plt.subplots(figsize=(12, 7))

methods = list(outlier_predictions.keys())
threats_real = (y_true == 1).sum()
outliers_detected = [pred.sum() for pred in outlier_predictions.values()]
true_positives = [((y_true == 1) & (pred == 1)).sum() for pred in outlier_predictions.values()]

x = np.arange(len(methods))
width = 0.25

bars1 = ax.bar(x - width, [threats_real]*len(methods), width, label='Ameaças Reais', color='#e74c3c', edgecolor='black', alpha=0.8)
bars2 = ax.bar(x, outliers_detected, width, label='Outliers Detectados', color='#3498db', edgecolor='black', alpha=0.8)
bars3 = ax.bar(x + width, true_positives, width, label='True Positives', color='#2ecc71', edgecolor='black', alpha=0.8)

ax.set_xlabel('Método de Detecção', fontsize=13, fontweight='bold')
ax.set_ylabel('Número de Amostras', fontsize=13, fontweight='bold')
ax.set_title('Comparação: Outliers Detectados vs Ameaças Reais', fontsize=15, fontweight='bold')
ax.set_xticks(x)
ax.set_xticklabels(methods, rotation=20, ha='right', fontsize=11)
ax.legend(fontsize=11, loc='upper left')
ax.grid(axis='y', alpha=0.3)

# Adicionar valores nas barras
for bars in [bars1, bars2, bars3]:
    for bar in bars:
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2., height,
                f'{int(height)}',
                ha='center', va='bottom', fontsize=9, fontweight='bold')

plt.tight_layout()
plt.savefig(os.path.join(OUTPUT_DIR, '03_outliers_comparison.png'), dpi=300, bbox_inches='tight')
plt.show()

print(f"✓ Visualização salva: {OUTPUT_DIR}/03_outliers_comparison.png")

## 8. Conclusões e Análise dos Resultados

### 8.1. Sumário Executivo

In [ ]:
print("="*80)
print("ANÁLISE FINAL DOS RESULTADOS")
print("="*80)

# Identificar melhor método por cada métrica
best_methods = {
    'Accuracy': df_results.loc[df_results['Accuracy'].idxmax()],
    'Precision': df_results.loc[df_results['Precision'].idxmax()],
    'Recall': df_results.loc[df_results['Recall'].idxmax()],
    'F1-Score': df_results.loc[df_results['F1-Score'].idxmax()]
}

print("\n🏆 MELHORES MÉTODOS POR MÉTRICA:")
print("-" * 80)
for metric, row in best_methods.items():
    print(f"{metric:12s}: {row['Método']:20s} ({row[metric]*100:.2f}%)")

# Análise geral
print("\n\n📊 ANÁLISE COMPARATIVA:")
print("-" * 80)

best_overall = df_results.iloc[0]
print(f"\n✅ MELHOR MÉTODO GERAL: {best_overall['Método']}")
print(f"   • F1-Score: {best_overall['F1-Score']*100:.2f}%")
print(f"   • Accuracy: {best_overall['Accuracy']*100:.2f}%")
print(f"   • Precision: {best_overall['Precision']*100:.2f}%")
print(f"   • Recall: {best_overall['Recall']*100:.2f}%")
print(f"   • Tempo de execução: {best_overall['Tempo (s)']:.2f}s")

worst_overall = df_results.iloc[-1]
print(f"\n❌ PIOR MÉTODO: {worst_overall['Método']}")
print(f"   • F1-Score: {worst_overall['F1-Score']*100:.2f}%")

# Insights
print("\n\n💡 INSIGHTS PRINCIPAIS:")
print("-" * 80)

print("\n1. EFICÁCIA NA DETECÇÃO DE AMEAÇAS:")
avg_precision = df_results['Precision'].mean()
avg_recall = df_results['Recall'].mean()
print(f"   • Precision média: {avg_precision*100:.2f}%")
print(f"   • Recall médio: {avg_recall*100:.2f}%")

if avg_precision > 0.5:
    print("   → Boa capacidade de identificar ameaças reais entre os outliers detectados")
else:
    print("   → Muitos falsos positivos - outliers não correspondem a ameaças")

if avg_recall > 0.5:
    print("   → Alta cobertura - maioria das ameaças foi detectada como outlier")
else:
    print("   → Baixa cobertura - muitas ameaças não foram detectadas")

print("\n2. TRADE-OFF PRECISION vs RECALL:")
precision_range = df_results['Precision'].max() - df_results['Precision'].min()
recall_range = df_results['Recall'].max() - df_results['Recall'].min()
print(f"   • Variação Precision: {precision_range*100:.2f}%")
print(f"   • Variação Recall: {recall_range*100:.2f}%")
print("   → Alguns métodos priorizam precision, outros recall")

print("\n3. PERFORMANCE COMPUTACIONAL:")
fastest = df_results.loc[df_results['Tempo (s)'].idxmin()]
slowest = df_results.loc[df_results['Tempo (s)'].idxmax()]
print(f"   • Mais rápido: {fastest['Método']} ({fastest['Tempo (s)']:.2f}s)")
print(f"   • Mais lento: {slowest['Método']} ({slowest['Tempo (s)']:.2f}s)")

print("\n" + "="*80)
print("✅ ANÁLISE COMPLETA!")
print("="*80)
print(f"\nResultados salvos em: {OUTPUT_DIR}/")
print(f"  • 01_exploratory_analysis.png")
print(f"  • 02_comparative_metrics.png")
print(f"  • 03_outliers_comparison.png")
print(f"  • 04_confusion_matrices.png")
print(f"  • evaluation_metrics.csv")

### 8.2. Considerações Finais

**Validação da Hipótese:**
- A análise demonstrou que técnicas de detecção de outliers podem ser eficazes para identificar ameaças cibernéticas em dados textuais
- Diferentes métodos apresentam diferentes trade-offs entre precision e recall
- Elliptic Envelope mostrou-se particularmente eficaz para este tipo de dado

**Aplicabilidade em Aprendizado Federado:**
- Estas técnicas podem ser usadas para detectar agentes maliciosos em ambientes federados
- A detecção de outliers pode identificar participantes que enviam atualizações suspeitas
- Importante considerar o equilíbrio entre detecção de ameaças e falsos positivos

**Próximos Passos:**
1. Testar com diferentes tamanhos de dataset
2. Avaliar em cenários de aprendizado federado real
3. Implementar ensembles de múltiplos métodos
4. Ajustar hiperparâmetros para otimizar F1-Score

---

**Referências:**
- Dataset: Text-Based Cyber Threat Detection (Kaggle)
- Scikit-learn Documentation
- Projeto: Mitigação de Ataques por Envenenamento em Aprendizado Federado